In [8]:
import pathlib
import os
import glob
import re
import csv
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
os.getcwd()

'/media/paulosschlogl/Paulo/pauloscchlogl/Genome_kmers'

In [3]:
dir_in = "Jelly/Bacteria/Pseudomonadota/Acidithiobacillia/CHR"
dir_out = "Jelly/Bacteria/Pseudomonadota/Acidithiobacillia/CHR"

In [4]:
file_names = open("jelly_sorted_uniq", 'r')

In [5]:
geno_ids = [name.strip() for name in file_names]

In [6]:
file_names.close()

In [7]:
len(geno_ids), geno_ids[:10]

(78,
 ['GCA_000020825.1',
  'GCA_000021485.1',
  'GCA_000175575.2',
  'GCA_000179815.2',
  'GCA_000214095.3',
  'GCA_000221025.1',
  'GCA_000423825.1',
  'GCA_000559045.1',
  'GCA_000709715.1',
  'GCA_000732185.1'])

In [12]:
header

'Assembly , Size(Mb)\n'

In [15]:
length_file = "Genomes_new/Bacteria/Pseudomonadota/Acidithiobacillia/Acidithiobacillia_len.csv"
# Mb = 1,000,000
lengths = defaultdict(int)
with open(length_file, "r") as csvfile:
    reader = csv.reader(csvfile)
    # escape the header of the csv file
    header = csvfile.readline()
    for line in reader:
        # line is a list
        geno_id = line[0].strip()
        # converting Mb to bp
        geno_len = round(float(line[1].strip()) * 1000000)
        lengths[geno_id] = lengths.get(geno_id, 0) + geno_len
lengths

defaultdict(int,
            {'GCA_016936895.1': 2241660,
             'GCA_001304445.1': 2283080,
             'GCA_001304215.1': 2616360,
             'GCA_001303835.1': 2826030,
             'GCA_018853595.1': 3501040,
             'GCA_001931655.1': 3503320,
             'GCA_008694225.1': 2876630,
             'GCA_000221025.1': 3237600,
             'GCA_000175575.2': 2987050,
             'GCA_001650235.3': 2950010,
             'GCA_001756725.1': 3160070,
             'GCA_001756745.1': 3143080,
             'GCA_001756675.1': 3122210,
             'GCA_018853615.1': 3019180,
             'GCA_018853575.1': 2900080,
             'GCA_018853545.1': 2860040,
             'GCA_010378095.1': 3165860,
             'GCA_003966655.1': 2921400,
             'GCA_003309025.1': 2716890,
             'GCA_018854635.1': 2899100,
             'GCA_018854615.1': 3142010,
             'GCA_018854535.1': 3133510,
             'GCA_018854695.1': 2917380,
             'GCA_018854545.1': 2944280,

In [16]:
filenames = glob.glob(f"{dir_in}/*.dump")

In [17]:
filenames[0]

'Jelly/Bacteria/Pseudomonadota/Acidithiobacillia/CHR/GCA_027446465.1_9.dump'

In [18]:
rg =  re.compile(r"(GC[AF]_\d+\.\d)")
rg

re.compile(r'(GC[AF]_\d+\.\d)', re.UNICODE)

In [20]:
def get_assembly_ids(string, regex):
    assm = regex.search(string).group()
    return assm

In [ ]:
#name = get_assembly_ids(filenames[0], rg)

In [61]:
def merge_jelly_dump_kmer(name, filenames, rg):
    grouped_files = []
    for filename in filenames:
        m = get_assembly_ids(filename, rg)
        if name == m:
            grouped_files.append(filename)
    dfs = [pd.read_table(filename, 
                         sep="\s+", 
                         names=["kmer", 
                                "observed"]) for filename in grouped_files]
    df = pd.concat(dfs)
    df.index = df["kmer"].str.len()
    df = df.sort_index(ascending=True).reset_index(drop=True)
    mask = df['kmer'].str.len() == 1
    length = df[mask]['observed'].sum()
    df['observed_freq'] = df["observed"] / length
    return df

In [62]:
df = merge_jelly_dump_kmer('GCA_027446465.1', filenames, rg)

In [63]:
df.head()

,kmer,observed,observed_freq
0,T,772009,0.235011
1,G,869480,0.264683
2,C,870215,0.264906
3,A,773286,0.235400
4,AT,214084,0.065170


In [54]:
mask = df['kmer'].str.len() == 1

In [64]:
df[mask]['observed'].sum(), df[mask]['observed_freq'].sum()

(3284990, 1.0)

In [68]:
mask2 = df['kmer'].str.len() == 10
df[mask2]['observed_freq'].sum()

0.9999917807968974

In [35]:
df["frequency"] = df["observed"] / lengths["GCA_027446465.1"]

In [36]:
df

,kmer,observed,frequency
0,T,772009,2.350089e-01
1,G,869480,2.646803e-01
2,C,870215,2.649040e-01
3,A,773286,2.353977e-01
4,AT,214084,6.516977e-02
...,...,...,...
1222041,CCGGCTATAT,4,1.217649e-06
1222042,CCGGCTATCA,4,1.217649e-06
1222043,CCGGCTATCC,9,2.739709e-06
1222044,CCGGCTATCT,3,9.132364e-07


In [29]:
#l = lengths["GCA_027446465.1"]

In [30]:
#772009/l, 869480/l, 870215/l, 773286/l

(0.23500891927598613,
 0.2646802759191725,
 0.2649040188492003,
 0.23539765359115014)

In [32]:
#sum([772009/l, 869480/l, 870215/l, 773286/l])

0.9999908676355092

In [43]:
# length - k + 1
#l2 = sum([772009, 869480, 870215, 773286])

In [60]:
#l2, l, l - l2, l2 -l

(3284990, 3285020, 30, -30)

In [44]:
#sum([772009/l2, 869480/l2, 870215/l2, 773286/l2])

1.0

In [38]:
#sum([772009, 869480, 870215, 773286]), l

(3284990, 3285020)

In [41]:
#772009/sum([772009, 869480, 870215, 773286]), 772009/l

(0.23501106548269554, 0.23500891927598613)

In [42]:
#0.23501106548269554 - 0.23500891927598613

(30, 2.146206709408016e-06)

In [70]:
#outfile = f"{dir_out}/{name}_k1_10.csv"

In [71]:
cnt = 0

for geno in geno_ids:
    df = merge_jelly_dump_kmer(geno, filenames, rg)
    out_file = f"{dir_out}/{geno}_k1_10.csv"
    df.to_csv(out_file, index=False)
    cnt += 1

In [72]:
print(f"Number of csv files created: {cnt}")

Number of csv files created: 78
